# Types of Inference Models


문서의 위아래로 탐색하려면 키보드의 방향키 키를 사용할 수 있습니다.<br />
이 노트북의 코드를 실행하려면 코드 블록을 선택하고 **Shift+Enter**를 누르십시오. <br />
코드 블록을 편집하려면 Enter 키를 누릅니다.

#### 노트북의 코드는 누적되어 사용됩니다.(정의된 변수는 노트북이 닫힐 때까지 계속 사용 가능) <br />
따라서 실행 순서를 위에서부터 아래로 진행하세요. 과정을 뛰어넘어 진행한다면 에러가 발생할 수 있습니다!


쥬피터 노트북 사용에 대한 추가 도움말을 보려면 위의 메뉴에서 도움말(Help) > 사용자 인터페이스 둘러보기(User Interface Tour)를 클릭하세요. <br />
또는 다음 사이트를 방문하여 살펴보세요. https://jupyter-notebook.readthedocs.io/en/stable/ui_components.html

여러분의 아이디어를 실험하고 테스트해보세요. 그것이 학습에 있어서 가장 빠른 방법 중 하나이기 때문입니다!

## 1. 기계 학습 모델은 무엇을 예측할 수 있습니까?

이전 세션에서는 이미지(입력)를 사용하여 모델을 학습시켰고, 어떤 클래스에 속하는지에 대한 예측 값을 반환(출력)하였습니다. 그것은 **분류(classification)** 의 한 형태입니다. 이미지 내의 카드의 위치를 결정하는 실습도 진행하였습니다. 그것은 **위치 식별(localization)** 의 한 형태입니다. 그리고 여러분 중 일부는 이미지의 각 개별 픽셀이 속한 클래스를 분류했습니다. 그것은 **이미지 세분화(image segmentation)** 의 한 형태였습니다.

일반적으로 기계 학습 모델은 특성 벡터를 입력으로 사용합니다. 컴퓨터 비전의 특성 벡터는 이미지에서 계산됩니다. 계산된 결과는 출력 집합을 예측하는 모델에 제공됩니다. 이러한 출력에는 다음이 포함될 수 있습니다:

- **이미지 분류**의 경우 이미지의 클래스 레이블
- **위치 식별**의 겨우 이미지의 클래스 레이블 및 경계 상자의 좌표
- **개체 감지**의 경우 감지된 개체에 대한 경계 상자(및 개체 인식을 위한 클래스 레이블) 목록
- **이미지 세분화**의 경우 이미지의 각 픽셀에 대한 클래스 레이블
- 출력에는 다른 클래스 레이블에 대한 확률 점수가 포함될 수도 있습니다.

오늘 세션에서는 컨볼루션 신경망(Convolutional Neural Network, CNN)을 소개한 다음, 실제 환경에서 분류 및 개체 감지를 위해 미리 학습된 모델을 사용하는 방법에 대해 살펴봅니다.
1. 컨벌루션 신경망은 무엇입니까?
2. Intel® OpenVINO™ 추론 엔진을 OpenCV와 함께 사용하는 방법
3. Intel Neural Compute Stick 2에서 추론하는 방법

## 컨볼루션 신경망 (CNN)

지난 세션에서는 kNN 및 SVM과 같은 기본 기계 학습 알고리즘을 사용했습니다. 최근 몇 년 동안, 컨볼루션 신경망(CNN)은 그 효과로 인해 큰 인기를 얻었습니다. CNN은 일련의 컨볼루션 필터로 구성된 신경망의 한 형태입니다. 그리고 네트워크는 훈련 과정을 통해 최적의 필터(가중치로도 알려져 있음)를 학습합니다.

**컨볼루션은 무엇입니까?**

간단히 말해서, 컨볼루션은 출력을 생성하기 위해 입력에서 수행되는 수학 연산입니다. 특성 추출의 한 형태로도 사용할 수 있습니다.

컨볼루션은 기본 이미지 처리에도 사용할 수 있습니다. 예를 들어 흐림, 선명화, 평균화 및 양방향 필터링이 있습니다.

CNN의 경우 모델은 훈련 중에 컨볼루션을 수행하면서 최적의 가중치를 학습합니다(수동으로 정의하는 대신). 각 컨볼루션에 대한 최적의 "가중치"를 계산하기 위해 그래디언트 하강과 같은 최적화 기술이 사용됩니다(오차의 역전파와 함께). 이 세션에서는 구현이나 수학에 대해 자세히 설명하지는 않겠지만, [자세히 알아보기](http://deeplearning.stanford.edu/tutorial/supervised/ConvolutionalNeuralNetwork/) 를 권장합니다. 

### 1.1 컨볼루션

인쇄회로기판(PCB) 사진에 몇 가지 컨볼루션 필터를 적용하여 컨볼루션의 개념을 어느 정도 시각화해 보겠습니다.  **cv2.filter2D** 는 OpenCV의 이미지 처리 기능으로 컨볼루션을 수행하는 데 사용할 수 있습니다. https://docs.opencv.org/2.4/doc/tutorials/imgproc/imgtrans/filter_2d/filter_2d.html

<img src="[Dataset]_Module22_images/convolutions.png" />

일부 컨볼루션 필터를 적용한 후 단어 및 회로 기판 패턴이 다른 것에 비해 얼마나 더 명확하고 선명해지는지 확인합니다. 이전 세션의 임계값이 생각나시나요? 컨볼루션은 입력 이미지에서 의미 있는 특성을 추출하는 데 사용할 수 있습니다. CNN의 경우 하나가 아닌 여러 컨볼루션이 일반적으로 사용됩니다.

몇 가지 코드를 살펴보겠습니다.

In [1]:
import cv2
import numpy as np

# 이미지를 읽기 위한 코드
image = cv2.imread("[Dataset]_Module22_images/pcb.png")

#  2D 컨볼루션을 실행하기 위해 회색조로 변환합니다.
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 회색조 이미지 표시
cv2.imshow("Original Greyscale Image",gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

컨볼루션 필터를 사용하여 다음과 같이 이미지를 흐리게 해보겠습니다. 원본과 비교해서 결과가 어떤가요?

In [2]:
kernel1 = np.ones((19,19),np.float32)/361      # 19x19 컨볼루션 필터
filter1 = cv2.filter2D(gray,-1,kernel1)

cv2.imshow("Original Greyscale Image",gray)
cv2.imshow("Filter1",filter1)
cv2.waitKey(0)
cv2.destroyAllWindows()

다른 필터를 사용해 봅시다

In [5]:
kernel2 = np.array((                        # 3x3 컨볼루션 필터
    [0, -1, 0],
    [-1, 4, -1],
    [0, -1, 0]), np.float32)*2
filter2 = cv2.filter2D(gray,-1,kernel2)

cv2.imshow("Original Greyscale Image",gray)
cv2.imshow("Filter2",filter2)
cv2.waitKey(0)
cv2.destroyAllWindows()

위의 코드에서 3x3 숫자 배열을 보셨습니까? 이것은 입력 이미지에 대한 수학적 연산을 수행하는 데 사용되는 컨볼루션 필터입니다. [자세히 보기](http://ufldl.stanford.edu/tutorial/supervised/FeatureExtractionUsingConvolution/#convolutions) 이와 같은 필터를 사용하여 관련 기능으로 이미지를 변환하는 방법을 확인할 수 있습니다. 링크의 예제는 결과로 생성되는 특성 벡터 출력이 입력보다 작게 표시됩니다. 그러나 예제에서 출력은 CV2.Filter2D가 자동으로 패딩을 추가하기 때문에 입력과 동일한 크기를 유지합니다.

1 개의 필터를 더 시도해 봅시다

In [4]:
kernel3 = np.array((
    [-2, -2, 0],
    [-2, 1, 2],
    [0, 2, 2]), dtype="int")           
filter3 = cv2.filter2D(gray,-1,kernel3)  

cv2.imshow("Original",gray)
cv2.imshow("Filter3",filter3)     # 출력된 이미지가 포토샵의 필터를 연상시키나요?
cv2.waitKey(0)
cv2.destroyAllWindows()

3x3 숫자 배열을 다시 보셨나요? CNN의 맥락에서 이러한 수치는 CNN이 모델 훈련 과정의 일부로 학습할 각 컨볼루션(CONV) 계층의 가중치로 알려져 있다.
컨볼루션 신경망은 일반적으로 일련의 컨볼루션 레이어(CONV)가 포함되며, 활성화 및 풀링 레이어와 함께 사용됩니다. 활성화 레이어는 일반적으로 모든 CONV 레이어와 풀링 레이어가 공간 치수를 줄이는 데 사용된 후에 적용됩니다. 과적합을 방지하기 위해 드롭아웃 레이어를 추가할 수도 있습니다. 자세한 사항은 이 세션의 범위를 벗어나지만 [더 읽기](http://cs231n.github.io/convolutional-networks/#layers) 를 권장합니다.

훈련이 완료되면 CNN은 다른 기계 학습 모델과 마찬가지로 예측을 할 수 있습니다.

## 2. Intel&reg; OpenVINO&trade; 추론 엔진

이제 사전 훈련된 CNN으로 주제를 전환해 보겠습니다. CNN에 대해 살펴보았고, 이제 사용법에 대해 알아봅시다. Intel&reg; OpenVINO&trade; Toolkit은 사용할 수 있는 일부 사전 훈련된 모델과 함께 제공됩니다.

모델 최적화를 사용하여 Caffe, MxNet, 텐서플로우와 같은 다른 프레임워크에서 학습된 모델을 변환할 수도 있습니다. 
이 툴킷은 현재 CPU, GPU, FPGA 또는 MYRIAD(NCS2)에서 최적화된 모델을 실행할 수 있도록 지원합니다.

### 모델 최적화 프로그램은 다음에서 사용할 수 있습니다:
%OPENVINO_INSTALL_DIR%/Intel/computer_vision_sdk/deployment_tools/model_optimizer 

모델 최적화 프로그램은 (파이썬 3.6을 설치한) cv 환경을 활성화하여 실행할 수 있습니다.  <br />
필요한 인수를 사용하여 **python mo.py**를 실행합니다.


### FP32 데이터 유형의 경우 CPU에 더 일반적으로 사용됩니다:
(%PATH_TO_MODEL% 및 %PATH_TO_OUTPUT% 대체)

python mo.py --input_model "%PATH_TO_MODEL%\squeezenet1.1.caffemodel" --output_dir "%PATH_TO_OUTPUT%/FP32" --data_type FP32

###  FP16 데이터 유형의 경우 NCS2에 더 일반적으로 사용됩니다.

(%PATH_TO_MODEL% 및 %PATH_TO_OUTPUT% 대체)

python mo.py --input_model "%PATH_TO_MODEL%\squeezenet1.1.caffemodel" --output_dir "%PATH_TO_OUTPUT%/FP16" --data_type FP16


원활한 시작을 위해서 일부 모델은 이미 "models" 하위 디렉터리에서 변환되었습니다.

보너스: FP16 및 FP32에 대해 [여기](https://www.quora.com/What-is-the-difference-between-FP16-and-FP32-when-doing-deep-learning) 에서 자세히 알아볼 수 있습니다.


### 2.1 분류를 위한 샘플 모델 (CPU에서 실행)

Intel® OpenVINO™ Python API에 대한 문서는 [온라인](https://software.intel.com/en-us/articles/OpenVINO-InferEngine#overview-of-inference-engine-python-api) 에서 확인가능 합니다.

Intel® OpenVINO™  설치 과정에서 분류를 수행하는 샘플 파이썬 파일도 함께 제공됩니다. 파일은 다음에서 찾을 수 있습니다.   %OPENVINO_INSTALL_DIR%/Intel/computer_vision_sdk/inference_engine/python_samples/classification_sample.py 

또한 다음 코드를 실행하고 결과를 표시해 볼 수 있습니다. 먼저 신경망이 로드되고 분류 결과(확률이 가장 높은 클래스 레이블로 시작)를 표시합니다.

### SqueezeNet을 사용하여 1000개의 다른 이미지 클래스 분류

이 모델은 1000개 클래스의 이미지를 인식하도록 훈련되었습니다. 프로젝트에 대한 자세한 내용은 [여기](https://arxiv.org/abs/1602.07360) 에서 확인할 수 있습니다.

다음으로 이 모델을 사용하여 몇 가지 분류를 해보겠습니다.

아래 예 외에도 images/dog.jpeg, images/hamster.jpeg, images/blucar.jpeg와 같은 몇 가지 다른 이미지도 사용해 볼 수 있습니다

In [6]:
!python classification_sample.py -i [Dataset]_Module22_images/cat.jpeg -m models/squeezenet1.1/FP32/squeezenet1.1.xml --labels models/squeezenet1.1/FP32/squeezenet1.1.labels

[ INFO ] Creating Inference Engine
[ INFO ] Loading network:
	models/squeezenet1.1/FP32/squeezenet1.1.xml
[ INFO ] Preparing input blobs
[ WARNING ] Image [Dataset]_Module22_images/cat.jpeg is resized from (1500, 2034) to (227, 227)
[ INFO ] Batch size is 1
[ INFO ] Loading model to the plugin
[ INFO ] Start inference (10 Asynchronous executions)
[ INFO ] Completed 1 Async request execution
[ INFO ] Completed 2 Async request execution
[ INFO ] Completed 3 Async request execution
[ INFO ] Completed 4 Async request execution
[ INFO ] Completed 5 Async request execution
[ INFO ] Completed 6 Async request execution
[ INFO ] Completed 7 Async request execution
[ INFO ] Completed 8 Async request execution
[ INFO ] Completed 9 Async request execution
[ INFO ] Completed 10 Async request execution
[ INFO ] Processing output blob
[ INFO ] Top 10 results: 
Image [Dataset]_Module22_images/cat.jpeg

classid probability
------- -----------
  cat     0.7695209
tabby cat 0.1498846
  cat    0.0361377
c

### 실습: ./images 폴더에서 사진을 하나 이상 더 분류하십시오!

In [7]:
#your code here

[ INFO ] Creating Inference Engine
[ INFO ] Loading network:
	models/squeezenet1.1/FP32/squeezenet1.1.xml
[ INFO ] Preparing input blobs
[ WARNING ] Image [Dataset]_Module22_images/bluecar.jpeg is resized from (1416, 2520) to (227, 227)
[ INFO ] Batch size is 1
[ INFO ] Loading model to the plugin
[ INFO ] Start inference (10 Asynchronous executions)
[ INFO ] Completed 1 Async request execution
[ INFO ] Completed 2 Async request execution
[ INFO ] Completed 3 Async request execution
[ INFO ] Completed 4 Async request execution
[ INFO ] Completed 5 Async request execution
[ INFO ] Completed 6 Async request execution
[ INFO ] Completed 7 Async request execution
[ INFO ] Completed 8 Async request execution
[ INFO ] Completed 9 Async request execution
[ INFO ] Completed 10 Async request execution
[ INFO ] Processing output blob
[ INFO ] Top 10 results: 
Image [Dataset]_Module22_images/bluecar.jpeg

classid probability
------- -----------
wagon, station wagon, wagon, estate car, beach waggo

더 많은 이미지를 사용해 보고 일부 이미지(특히 배경에 더 많은 개체가 있는 이미지)가 잘못 분류되는 이유를 살펴보십시오.

다음으로, Edge 디바이스에서 추론 작업 속도를 높이기 위해 설계된 휴대용 USB 장치 인 NCS2에서 동일한 모델을 실행하는 방법을 알아보겠습니다. 노트북에서도 사용할 수 있습니다.


### 2.2 분류를 위한 샘플 모델 (NCS2에서 실행)

Neural Compute Stick은 대규모 네트워크에 대한 추론을 쉽게 하기 위한 것입니다. CPU 대신 NCS2에서 추론을 실행하는 방법을 알아보겠습니다.

먼저 NCS2를 컴퓨터의 USB 포트에 연결해야 합니다. USB3.0 포트가 있는 경우 빠른 데이터 전송 속도를 위해 포트를 사용합니다. 그런 다음 아래 코드를 실행합니다.

위의 코드와 얼마나 다릅니까?

CPU 대신 NCS2를 사용하도록 지정하기 위해 **-d MIRID**를 추가했습니다.(Myriad는 실제로 NCS2 안에 있는 강력한 칩의 이름입니다.) 그리고 모델 경로도 FP32 디렉토리가 아닌 FP16 디렉토리로 업데이트하였습니다. 그 이유는 CPU가 FP32를 사용하는 경향이 있는 반면 NCS2는 **FP16**만 지원하기 때문입니다.

In [8]:
!python classification_sample.py -d MYRIAD -i [Dataset]_Module22_images/cat.jpeg -m models/squeezenet1.1/FP16/squeezenet1.1.xml --labels models/squeezenet1.1/FP16/squeezenet1.1.labels
#In case not using NCS2 please replace "MYRIAD" with "CPU"

[ INFO ] Creating Inference Engine
[ INFO ] Loading network:
	models/squeezenet1.1/FP16/squeezenet1.1.xml
[ INFO ] Preparing input blobs
[ WARNING ] Image [Dataset]_Module22_images/cat.jpeg is resized from (1500, 2034) to (227, 227)
[ INFO ] Batch size is 1
[ INFO ] Loading model to the plugin


Traceback (most recent call last):
  File "classification_sample.py", line 176, in <module>
    sys.exit(main() or 0)
  File "classification_sample.py", line 134, in main
    exec_net = ie.load_network(network=net, device_name=args.device)
  File "ie_api.pyx", line 403, in openvino.inference_engine.ie_api.IECore.load_network
  File "ie_api.pyx", line 442, in openvino.inference_engine.ie_api.IECore.load_network
RuntimeError: Can not init Myriad device: NC_ERROR


### 실습: ./images 폴더에서 사진을 하나 이상 더 분류하십시오!

In [9]:
#your code here

[ INFO ] Creating Inference Engine

Traceback (most recent call last):
  File "classification_sample.py", line 176, in <module>
    sys.exit(main() or 0)
  File "classification_sample.py", line 134, in main
    exec_net = ie.load_network(network=net, device_name=args.device)
  File "ie_api.pyx", line 403, in openvino.inference_engine.ie_api.IECore.load_network
  File "ie_api.pyx", line 442, in openvino.inference_engine.ie_api.IECore.load_network
RuntimeError: Can not init Myriad device: NC_ERROR



[ INFO ] Loading network:
	models/squeezenet1.1/FP16/squeezenet1.1.xml
[ INFO ] Preparing input blobs
[ WARNING ] Image [Dataset]_Module22_images/dog.jpeg is resized from (717, 800) to (227, 227)
[ INFO ] Batch size is 1
[ INFO ] Loading model to the plugin


CPU와 NCS2에서 생성된 확률은 부동 소수점이 지원/계산되는 방식으로 인해 일부 차이가 있을 수 있습니다.  NCS2에는 FP16 모델을, CPU에는 FP32 모델을 사용했다는 것을 기억하십시오. 이 두 모델은 완전히 동일하지 않습니다.모델 하위 디렉터리에서 모델 파일 크기를 살펴보면 FP16 모델의 .bin 파일이 FP32 모델의 .bin 파일보다 훨씬 작은 경향이 있습니다.

CPU와 NCS2의 결과를 비교하면 이 네트워크는 10-20ms 이내에 빠르게 실행되는 소규모 네트워크이기 때문에 네트워크의 실행 시간이 약간 개선될 수 있습니다. 그러나 나중에 보겠지만, 현재 CPU를 통해 지원되지 않는 NCS2에서 실행할 수 있는 Intel 사전 교육 모델이 있습니다. 자세히 살펴보기 전에 classification_sample.py 내부의 코드를 살펴보도록 하겠습니다.

**모델 사용 후 코드를 더 자세히 살펴보겠습니다.**

코드가 원래 효율성을 높이기 위해 C ++로 작성 되었기 때문에 파이썬 패키지를 만들었습니다. openvino.inference_engine 패키지의 IEPlugin와 IENetwork 모듈 입니다.

우선 classification_sample.py를 열어 코드를 이해할 수 있습니다. 일반적인 아이디어는 다음과 같습니다:
1. IEPlugin(CPU / Meriod / GPU / FPGA)을 통해 하드웨어를 초기화합니다.
2. iEnetwork를 통해 네트워크를로드합니다.
3. 입력 이미지를 필요한 형식으로 전처리합니다.
4. exec_net.infer를 통해 예측을합니다.
5. 출력을 처리/표시합니다.
6. 메모리를 해제합니다. 즉, del plugin, del net 및 del exec_net 키워드는 파이썬 애플리케이션에서는 많이 사용되지 않지만, NCS2에 대한 부하를 해제하기 위해서는(주피터 노트북에서 실행될 때) 필요합니다. 명령줄을 통해 직접 실행될 때 스크립트가 실행을 완료하면 자동 릴리스가 있을 수 있습니다.   

계속하기 전에 "classification_sample.py"를 열어 코드를 간단히 읽어보세요.

### 2.3 파이썬 클래스를 사용한작업 간소화

종종 우리는 하위 수준의 코드를 반복적으로 구현하고 싶지 않습니다. 이전 세션을 기억한다면, sklearn 패키지에서 모델을 로드하고 나면 **model.proxed()** 라는 멋진 메소드가있었습니다.

마찬가지로 CNN 모델을 로드한 다음 model.pedicted와 비슷한 것을 사용하여 예측을 생성할 수 있습니까?

네, 할 수 있습니다.

https://github.com/simpledevelopments/OpenVINO-Python-Utils 의 지원 클래스를 사용합시다. 
패키지는 인텔 ® OpenVino ™ 툴킷의 2018R5 릴리스에서 작동하도록 테스트되었습니다. 
추론 엔진 파이썬 API가 변경될 수 있기 때문에, API가 변경되면 코드를 다시 업데이트해야 합니다.

In [56]:
import cv2
import numpy as np
from utils.opv import OpvModel      # OpenVINO 2021.2 버젼 지원 클래스

In [57]:
# "Can not init USB device: NC_DEVICE_NOT_FOUND"라는 오류가 발생하면, 
#  NCS2가 USB 포트에 연결되어 있는지 확인하십시오.
#
# 더 이상 추론을 만들기 위해 mymodel이 필요하지 않으면, 
# mymodel.ClearMachine()을 실행하여 장치에서 응용 프로그램을 해제하십시오.

mymodel = OpvModel("squeezenet1.1", device="CPU", fp="FP16", ncs=1, debug=True) 
# NCS2를 사용하지 않는 경우 "MYRIAD"를 "CPU"로 대체하십시오.

[INFO] Model squeezenet1.1 Loaded and Ready on NCS device 1


In [58]:
predictions = mymodel.Predict(cv2.imread("[Dataset]_Module22_images/cat.jpeg"))

	[INFO] Image resized from (1500, 2034) to (227, 227)


In [59]:
# 더 이상 추론을 만들기 위해 mymodel이 필요하지 않으면, 
# mymodel.ClearMachine()을 실행하여 장치에서 응용 프로그램을 해제하십시오.
# 또는 파이썬 응용 프로그램이 닫힐 때 자동 정리를 기다립니다.
#  (주피터 노트북의 경우, 파일 > 닫기 및 중지를 사용할 때 발생합니다.)

#mymodel.ClearMachine()

In [60]:
def displayresults(predictions, labels=None):
    if (labels is None):                         # 레이블이 제공되지 않는 경우 숫자 레이블을 사용합니다.
        labels = np.arange(0,1000).astype("str")
    predictions = predictions.reshape(1000)
    top5 = predictions.argsort()[-5:][::-1]
    top5results = np.column_stack([predictions[top5], labels[top5]])
    for (conf, label) in top5results:
        print(str(round(float(conf)*100,1))+"% chance: "+label)

displayresults(predictions,mymodel.labels)

77.0% chance: Persian cat
15.0% chance: tabby, tabby cat
3.6% chance: tiger cat
1.9% chance: lynx, catamount
0.9% chance: Egyptian cat


NCS2에서도 실행된 섹션 2.2와 결과를 비교하십시오. 결과가 일치해야 합니다.

### 참고: NumPy 배열에 대한 수정
NumPy 배열을 정렬하고 하위 설정하는 방법을 알아보려면 아래를 참조하십시오!

In [61]:
import numpy as np

a = np.array([1, 2, 3, 4, 5, 2, 1])
print(a)

[1 2 3 4 5 2 1]


In [62]:
np.sort(a)
print (np.sort(a))

[1 1 2 2 3 4 5]


In [63]:
np.flip(np.sort(a))
print (np.flip(np.sort(a)))

[5 4 3 2 2 1 1]


In [64]:
np.flip(np.sort(a))[:2]
print (np.flip(np.sort(a))[:2])

[5 4]


In [65]:
print(a.argsort()[-2:][: :-1]) # 값 5와 4의 인덱스 가져오기

[4 3]


In [66]:
a[a.argsort()[-2:][: :-1]]
print(a[a.argsort()[-2:][: :-1]])

[5 4]


**2.3.1 그 결과는 어떻게 깔끔하게 정리되었는가?**

그 코드가 무엇을 했는지 궁금하십니까? 예측부터 차근차근 진행합시다.

우리는 1000 개의 레코드를 기대하고 있으며 각 레코드는 해당 클래스에 속한 이미지의 확률을 표시합니다. 그러나 Predictions.shape은 우리의 배열을 (1,1000,1,1) 형태로 보여줍니다. 따라서 첫 번째 단계는 1D 배열로 재구성하는 것입니다.

In [67]:
print(predictions.shape)

(1, 1000, 1, 1)


In [68]:
predictions = predictions.reshape(1000)

In [69]:
print(predictions.shape)

(1000,)


**2.3.2 상위 5 개 결과에 대한 배열 인덱스를 어떻게 얻을 수 있습니까?**

NumPy에 대해 자세히 알아 보려면 https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.argsort.html 을 방문하십시오.

In [70]:
predictions.argsort()[-5:][::-1]

array([283, 281, 282, 287, 285], dtype=int64)

**2.3.3 이러한 "상위" 예측 각각에 대한 확률은 얼마입니까?**

이 값은 십진 형식으로 표시됩니다(1은 100% 신뢰도를 의미하고 0.1은 10% 신뢰도를 의미함).

In [71]:
predictions[predictions.argsort()[-5:][::-1]]

array([0.7695209 , 0.14988461, 0.0361377 , 0.01878894, 0.00871808],
      dtype=float32)

**2.3.4 이러한 예측에 대한 클래스 레이블은 무엇입니까?**


이 특정 예제의 경우, 모델을 로드할 때, 다른 클래스의 레이블도 squeezenet1.1.labels (models/squeezenet1.1/FP16 subdirectory 하위 디렉토리 내부)에서 로드되었으므로 mymodel.labels를 사용하여 액세스할 수 있습니다. 처음 5 개의 레코드를 인쇄 해 봅시다. squeezenet1.1.labels 파일과 교차 확인할 수 있습니다.

In [72]:
print(mymodel.labels[:5])

['tench, Tinca tinca' 'goldfish, Carassius auratus'
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias'
 'tiger shark, Galeocerdo cuvieri' 'hammerhead, hammerhead shark']


**2.3.5 좋습니다. 하지만 어떻게 예측에 라벨을 적용할 수 있을까요?**

다음은 상위 5개 결과입니다!

In [73]:
mymodel.labels[predictions.argsort()[-5:][::-1]]

array(['Persian cat', 'tabby, tabby cat', 'tiger cat', 'lynx, catamount',
       'Egyptian cat'], dtype='<U121')

**2.3.6 이제 확률과 결과를 어떻게 배열로 결합할 수 있을까요?**

In [74]:
top5 = predictions.argsort()[-5:][::-1]
np.column_stack([    predictions[top5],    mymodel.labels[top5]     ])

array([['0.7695209', 'Persian cat'],
       ['0.14988461', 'tabby, tabby cat'],
       ['0.036137696', 'tiger cat'],
       ['0.018788941', 'lynx, catamount'],
       ['0.008718076', 'Egyptian cat']], dtype='<U121')


**2.3.7 여기서 모든 것을 다시 정리해봅시다.**

In [75]:
def displayresults(predictions, labels=None):
    if (labels is None):                         # 레이블이 제공되지 않는 경우 숫자 레이블을 사용합니다.
        labels = np.arange(0,1000).astype("str")
    predictions = predictions.reshape(1000)
    top5 = predictions.argsort()[-5:][::-1]
    top5results = np.column_stack([predictions[top5], labels[top5]])
    for (conf, label) in top5results:
        print(str(round(float(conf)*100,1))+"% chance: "+label)
displayresults(predictions,mymodel.labels)

77.0% chance: Persian cat
15.0% chance: tabby, tabby cat
3.6% chance: tiger cat
1.9% chance: lynx, catamount
0.9% chance: Egyptian cat


### 실습: 모든 것을 종합해서, 다른 이미지에 대해 예측을 하세요.

In [76]:
#your code here

	[INFO] Image resized from (1416, 2520) to (227, 227)
29.4% chance: beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon
21.2% chance: grille, radiator grille
20.3% chance: minivan
13.0% chance: racer, race car, racing car
10.4% chance: sports car, sport car


잘 했어요! 사전 학습된 SqueezeNet1.1 및 OpenVINO를 사용하여 분류를 성공적으로 수행했습니다. 시간을 내어 모델을 로드하고 분류를 수행하고 결과를 표시하는 프로세스에 대해 생각해 보십시오. 학생 활동 가이드에 메모를 작성할 수 있습니다.

## 3. 추론 모델 라이브러리

추론 모델에는 다양한 유형이 있다는 것을 기억하십시오. 일부는 이미지 분류를 하고 일부는 객체 감지하고 일부는 이미지 세분화를 수행합니다. Intel은 또한 Intel® OpenVINO™ 설치와 함께 사전 학습된 추론 모델 세트를 제공합니다. 사전 학습된 모델에 대한 자세한 내용은 다음 웹 사이트에서 확인할 수 있습니다.
https://software.intel.com/en-us/openvino-toolkit/documentation/pretrained-models

**모델을 IR 형식으로 최적화합니다**<br />
섹션 2의 시작 부분에서 모델 최적화 도구에 대해 간략하게 이야기했습니다. 학습 된 모델이 인텔 ® OpenVino ™ 추론 엔진에서 실행되기 전에 중간 표현 (IR) 형식으로 최적화되어야합니다. 이러한 방식으로 CAFFE, MXNET 및 Tensorflow와 같은 프레임 워크에서 학습 된 모델은 툴킷을 사용하여 효율적으로 실행할 수있는 IR 형식으로 변환 할 수 있습니다. 자세한 내용은 https://software.intel.com/en-us/articles/OpenVINO-ModelOptimizer 를 참조하십시오.

**IR 형식으로 사전 학습된 모델**<br />
좋은 소식은 사전 학습된 인텔 모델의 경우 이미 IR 형식으로 변환되어 즉시 사용할 준비가 되어있습니다. 자세한 내용은 %OPENVINO_INSTALL_DIR%/Intel/computer_vision_sdk/deployment_tools/intel_models 에서 찾을 수 있습니다.

이러한 모델을 사용하려면 해당 하위 폴더를 이 노트북이 있는 모델 하위 디렉터리에 복사하면 됩니다.

## 4. 인텔 모델 "라이브러리"의 모델을 사용한 얼굴 검출



디지털 카메라는 사람들의 얼굴을 어떻게 감지합니까? 우리도 그런 일을 할 수 있을까요?

<img src="[Dataset]_Module22_images/friendsfaces.png" style="width:400px; float:left;" />
<div style="clear:both;"></div>

이를 위해 **face-detection-adas-0001** 모델을 사용해 보십시오!

### 실습: 모델 설명서를 참조하여 모델의 입력 및 출력을 기록하십시오. 

이 모델에 대한 설명서는 %OPENVINO_INSTALL_DIR%/Intel/computer_vision_sdk/deployment_tools/intel_models/face-detection-adas-0001/description/face-detection-adas-0001.html 에서 확인할 수 있습니다.

네트워크(모델)는 [1, 1, N, 7] 형태로 블롭을 출력하며, 여기서 N은 검출된 경계 상자의 수입니다. 각 탐지에 대해 설명은 [image_id, label, conf, x_min, y_min, x_max, y_max] 형식을 갖습니다.(7 개의 변수를 참조하십시오. 이는 [1,1,N,7]의 7을 설명합니다.)

**intel_models의 "face-detection-adas-0001"폴더를 모델 폴더로 복사 한 다음 시작해 봅시다!**

In [78]:
import cv2
import numpy as np
from utils.opv import OpvModel

mymodel2 = OpvModel("face-detection-adas-0001", device="CPU", fp="FP16", ncs=1) 
# NCS2를 사용하지 않는 경우 "MYRIAD"를 "CPU"로 대체하십시오.

In [79]:
friends = cv2.imread("[Dataset]_Module22_images/friends.jpeg")
friends = cv2.resize(friends,(1200,710))     # 원본이 상당히 크기 때문에 이미지 크기를 줄입니다.
predictions = mymodel2.Predict(friends)

In [80]:
predictions.shape

(1, 1, 200, 7)

획득한 모양을 문서에 표시된 출력 모양과 비교합니다. 모양은 동일해야 합니다!

얼마나 많은 예측이 있는지 알 수 있습니까?

In [81]:
# 답변 : 

이제 예측 변수에 무엇이 포함되어 있는지 궁금하십니까? 

### 실습: 예측의 세 개 이상의 결과를 나열합니다.  이들의 레이블은 [image_id, label, conf, x_min, y_min, x_max, y_max] 입니다.

In [82]:
#your code here

[[[[0.         1.         1.         0.7941595  0.39807257 0.84158474
    0.5066318 ]
   [0.         1.         0.99999964 0.35023394 0.4601197  0.39078203
    0.5425858 ]
   [0.         1.         0.995613   0.5246759  0.4436372  0.5696493
    0.5426169 ]]]]


이제 경계 상자를 그릴 때입니다! 아래 코드를 사용해 보십시오!

In [83]:
def DrawBoundingBoxes(predictions, image, conf=0):
    canvas = image.copy()                             # 원본 이미지를 수정하는 대신 복사합니다
    predictions_1 = predictions[0][0]                 # 하위 집합 데이터 프레임
    confidence = predictions_1[:,2]                   # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
    topresults = predictions_1[(confidence>conf)]     # 임계값보다 큰 conf 값을 가진 예측만 선택
    (h,w) = canvas.shape[:2]                         
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)   # 사각형 만들기
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin), # 텍스트 포함
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    cv2.putText(canvas, str(len(topresults))+" face(s) detected", (50,50),    # 텍스트 포함
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    return canvas

cv2.imshow("Friends2",DrawBoundingBoxes(predictions,friends))
cv2.waitKey(0)
cv2.destroyAllWindows()

우리가 너무 많은 상자를 그리는 것 같습니다. 200개의 경계 상자를 감지한 것 같습니다. 그러나 모든 경계 상자가 얼굴이 될 수 있는 것은 아닙니다. 어떤 것이 유효한 예측인지 확인하려면 확률 점수를 우리가 설정한 임계 값과 비교해야 합니다

### 실습: 50% 의 신뢰도를 임계값으로 사용합니다.

분류 예제에 대한 결과를 표시한 방법을 기억하고 아래 코드가 어떻게 작동하는지 비교하십시오.

In [84]:
def DrawBoundingBoxes(predictions, image, conf=0.5):
    canvas = image.copy()                             # 원본 이미지를 수정하는 대신 복사합니다
    predictions_1 = predictions[0][0]                 # 하위 집합 데이터 프레임
    confidence = predictions_1[:,2]                    # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
    topresults = predictions_1[(confidence>conf)]      # 임계값보다 큰 conf 값을 가진 예측만 선택
    (h,w) = canvas.shape[:2]                           # 이미지 높이에 따라 변수 h 및 w 설정
    
    #
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)  # 사각형 만들기
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin), # 텍스트 포함
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    cv2.putText(canvas, str(len(topresults))+" face(s) detected", (50,50), # 텍스트 포함
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    return predictions_1, canvas

predictions_1,x = DrawBoundingBoxes(predictions,friends)
cv2.imshow("Friends2",x)
cv2.waitKey(0)
cv2.destroyAllWindows()

여러분은 무엇을 볼 수 있습니까?  상자 안에 우리 친구들의 얼굴만 넣어져 있는 것을 볼 수 있습니다!


이제 코드에 대한 이해도를 테스트할 시간입니다.

### 실습: confidence 변수를 출력합니다. 이 변수에는 무엇이 포함되어 있습니까? 이미지의 네 친구를 가리키는 신뢰 수준을 확인할 수 있습니까?

In [85]:
# your code/answer here

[1.         0.99999964 0.995613   0.9652484  0.01954742 0.01897001
 0.01794284 0.01770019 0.01764668 0.01762706 0.01760947 0.01733034
 0.01699277 0.01693124 0.01667533 0.01658139 0.01621212 0.01598535
 0.01597153 0.01589354 0.01587604 0.01585953 0.01573428 0.01567845
 0.01560695 0.01541477 0.01540966 0.01534926 0.01514687 0.01509106
 0.01504166 0.01500521 0.01474105 0.01465906 0.01453096 0.01449704
 0.01445426 0.01443859 0.01428939 0.01418993 0.01384502 0.01382889
 0.01380085 0.01380065 0.01364383 0.01363705 0.01353545 0.01353054
 0.01335032 0.01334383 0.01331454 0.01328486 0.013275   0.01315909
 0.01310863 0.01310847 0.01307227 0.01304318 0.01303701 0.01303307
 0.01297615 0.01292788 0.01288899 0.01287343 0.01286598 0.0128212
 0.01280037 0.01278295 0.01277385 0.01276268 0.0127204  0.01269993
 0.01265149 0.01264626 0.01263556 0.01261097 0.01256746 0.0125375
 0.01253317 0.01247998 0.01228532 0.01223279 0.01223096 0.01219887
 0.01218056 0.01211949 0.01201286 0.01199868 0.01191019 0.011898

### 실습: topresults 변수를 출력합니다. 이 변수에는 무엇이 포함되어 있습니까? 
[image_id, label, conf, x_min, y_min, x_max, y_max] 를 다시 식별하십시오.

In [86]:
# your code/answer here

[[0.         1.         1.         0.7941595  0.39807257 0.84158474
  0.5066318 ]
 [0.         1.         0.99999964 0.35023394 0.4601197  0.39078203
  0.5425858 ]
 [0.         1.         0.995613   0.5246759  0.4436372  0.5696493
  0.5426169 ]
 [0.         1.         0.9652484  0.14379883 0.32845277 0.1896973
  0.41683376]]


### 실습: detection [3:7] 을 출력합니다.
무엇을 의미합니까?

In [93]:
# your code/answer here

array([0.7941595 , 0.39807257, 0.84158474, 0.5066318 ], dtype=float32)

### 실습: variable box를 출력합니다. 
무엇을 의미합니까?

In [94]:
# your code/answer here

[ 952.99136639  282.6315251  1009.90169048  359.70857203]


그런 다음 경계 상자와 텍스트 레이블을 그릴 수 있습니다.

### 실습: 자신의 설정으로 경계 상자를 다시 그립니다.
이번에는 상자의 색상과 두께, 글꼴의 크기와 유형을 변경합니다.

In [95]:
def DrawBoundingBoxes(predictions, image, conf=0.5):
    canvas = image.copy()                              # 원본 이미지를 수정하는 대신 복사합니다
    predictions_1 = predictions[0][0]                  # 하위 집합 데이터 프레임
    confidence = predictions_1[:,2]                   # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
    topresults = predictions_1[(confidence>conf)]     # 임계값보다 큰 conf 값을 가진 예측만 선택
    (h,w) = canvas.shape[:2]                          # 이미지 높이에 따라 변수 h 및 w 설정
    
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (100, 150, 255), 2)   # 사각형 만들기
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin), # 텍스트 포함
            cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 230,40), 2)
    cv2.putText(canvas, str(len(topresults))+" friend(s) detected. It's time to party!", (50,50), # 텍스트 포함
            cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 10,110), 2)
    return predictions_1, canvas

predictions_1,x = DrawBoundingBoxes(predictions,friends)
cv2.imshow("Friends2",x)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 실습: 모델을 사용하여 다른 이미지에 경계 상자를 분류하고 그립니다!
인물의 옆모습, 익살스러운 얼굴 등을 찍어 보세요. 미리 훈련된 모델이 얼굴을 찾는 능력에 대해 무엇을 알 수 있습니까?

In [96]:
#your code here

## 축하합니다! 여러분은 이제 막 첫 얼굴 탐지기를 만들었습니다!

## 이제 여러분이 손을 써야할 시간입니다! 

방금 2 개의 매우 강력한 훈련 된 모델을 사용했습니다. 하나는 1000개의 객체 클래스들 사이의 이미지 분류를 수행하고, 하나는 얼굴 감지를 수행합니다.

Intel® Distribution of OpenVINO Toolkit에서 제공하는 다양한 사전 교육 모델을 살펴보면서 intel_models 하위 디렉토리와 각 모델의 설명 하위 폴더를 살펴보십시오.문서는 각 모델이 예측할 수있는 것에 대한 더 나은 아이디어를 제공 할 것입니다. 예측을 완료하면 기본 이미지 처리 기술을 사용하여 결과를 표시합니다.

어떤 것을 구축하든, 목적과 목표를 염두에 두고 다양한 접근 방식을 생각해 보십시오. 기계 학습 알고리즘이 잘못 분류하고 위험을 완화하는 방법을 계획할 때 발생할 수 있는 영향도 명심하십시오.
예를 들어 실제로 얼굴이 없을 때 얼굴을 감지하면 최악의 결과는 무엇입니까? 아니면 실제로 얼굴이 있을 때 감지하지 못하는 경우의 결과는 무엇입니까? 오류가 발생할 가능성이 있는 다른 시나리오와 마찬가지로 실제 솔루션의 설계를 다른 기술 또는 하드웨어 센서로 보완하십시오.

### 도전과제 1: 섹션 1에서 배운 것을 사용하여 images/dog.jpeg 를 흐르게 합니다.

흐림 효과를 더 많이 흐리게 만드는 방법을 알고 있습니까?

In [98]:
#your code here

### 도전과제 2: 섹션 2.3 코드를 수정하여 상위 5개 대신 상위 3개 결과를 제공합니다.

In [100]:
mymodel = OpvModel("face-detection-adas-0001", device="CPU", fp="FP16", ncs=1, debug=True)
# (self, model_name, device, fp="FP32", debug=False, ncs=1)
# NCS2를 사용하지 않는 경우 "MYRIAD"를 "CPU"로 대체하십시오.

Loaded Machine Released
[INFO] Model face-detection-adas-0001 Loaded and Ready on NCS device 1


In [ ]:
#your code here

### 도전과제 3: models/squeezenet1.1/FP16/squeezenet1.1.labels 에서 레이블을 numpy 배열로 로드합니다. 파일 내부의 첫 번째 레이블은 무엇입니까?

파일에서 레이블을 읽고 numpy 배열로 로드하는 방법은 무엇입니까? <br />
힌트: 문제가 발생할 경우 utils/opv.py내에서 어떻게 수행되는지 확인할 수 있습니다.

In [102]:
#your code here

['tench, Tinca tinca' 'goldfish, Carassius auratus'
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias'
 'tiger shark, Galeocerdo cuvieri' 'hammerhead, hammerhead shark'
 'electric ray, crampfish, numbfish, torpedo' 'stingray' 'cock' 'hen'
 'ostrich, Struthio camelus' 'brambling, Fringilla montifringilla'
 'goldfinch, Carduelis carduelis'
 'house finch, linnet, Carpodacus mexicanus' 'junco, snowbird'
 'indigo bunting, indigo finch, indigo bird, Passerina cyanea'
 'robin, American robin, Turdus migratorius' 'bulbul' 'jay' 'magpie'
 'chickadee' 'water ouzel, dipper' 'kite'
 'bald eagle, American eagle, Haliaeetus leucocephalus' 'vulture'
 'great grey owl, great gray owl, Strix nebulosa'
 'European fire salamander, Salamandra salamandra'
 'common newt, Triturus vulgaris' 'eft'
 'spotted salamander, Ambystoma maculatum'
 'axolotl, mud puppy, Ambystoma mexicanum' 'bullfrog, Rana catesbeiana'
 'tree frog, tree-frog'
 'tailed frog, bell toad, ribbed toad, ta

### 도전과제 4: 분류 모델을 사용하여 images/pcb.png 를 분류합니다. 그것은 인쇄 회로 기판 (PCB)로 분류됩니까? 왜 그렇게 생각하십니까?

In [103]:
#your code here

[ INFO ] Creating Inference Engine
[ INFO ] Loading network:
	models/squeezenet1.1/FP32/squeezenet1.1.xml
[ INFO ] Preparing input blobs
[ WARNING ] Image [Dataset]_Module22_images/pcb.png is resized from (480, 640) to (227, 227)
[ INFO ] Batch size is 1
[ INFO ] Loading model to the plugin
[ INFO ] Start inference (10 Asynchronous executions)
[ INFO ] Completed 1 Async request execution
[ INFO ] Completed 2 Async request execution
[ INFO ] Completed 3 Async request execution
[ INFO ] Completed 4 Async request execution
[ INFO ] Completed 5 Async request execution
[ INFO ] Completed 6 Async request execution
[ INFO ] Completed 7 Async request execution
[ INFO ] Completed 8 Async request execution
[ INFO ] Completed 9 Async request execution
[ INFO ] Completed 10 Async request execution
[ INFO ] Processing output blob
[ INFO ] Top 10 results: 
Image [Dataset]_Module22_images/pcb.png

classid probability
------- -----------
site, website, internet site, site0.2059917
sunblock, sun blocke

### 도전과제 5: 섹션 4에서 모델에 대해 1 예측을 수행하는 데 걸리는 시간을 계산하십시오.

In [104]:
import time
def wastetime():
    for i in range(0,10000000):
        testing = 1
        
# 시간차를 계산하기위한 샘플 코드
a = time.time()
wastetime()
b = time.time()
print(str(b-a) + " seconds")

0.16396164894104004 seconds


In [105]:
#your code here

(1492, 2520, 3)
(1, 1, 200, 7)
0.023909568786621094 seconds


### 도전과제 6: 웹캠을 통해 실시간 얼굴 탐지를 수행하는 응용 프로그램을 만듭니다.

힌트: 섹션 4에서 배운 내용과 이전 세션의 웹캠을 통한 비디오 캡처에 대해 이미 알고 있는 내용을 활용하십시오.

In [106]:
#your code here

### 도전과제 7: 실시간 얼굴 감지가 화면에서 약간 느리게 보입니까? 탐지를 더 빠르게 하는 1가지 이상의 방법을 생각할 수 있습니까?

힌트: 이미지가 클수록 처리가 더 많이 필요합니다. 또한 모든 모델이 똑같이 만들어지는 것은 아닙니다. 다른 방법을 생각하고 그것을 시도해보십시오!

In [107]:
#Try yourself!

### 도전과제 8: 동시에 2 개의 NCS 장치를 사용할 수 있습니까?

친구로부터 NCS 장치를 하나 더 빌리면 얼굴 감지와 분류 모델을 동시에 실행할 수 있습니다. 첫 번째 모델의 경우 NCS 플래그를 1로 설정하고 두 번째 모델의 경우 NCS 플래그를 2로 설정합니다. 또한 디버그 플래그를 true로 설정하여 출력을 볼 수도 있습니다.

예를들어, <br />
mymodel = OpvModel("face-detection-retail-0004", device="MYRIAD", fp="FP16", ncs=1, debug = True) <br />
mymodel2 = OpvModel("squeezenet1.1", device="MYRIAD", fp="FP16", ncs=2, debug = True)

In [108]:
# Try!

## 도전과제 9: NCS2를 만져보세요. 뜨거움을 느끼나요?

NCS2를 해제하려면 해당 메모리를 해제해야합니다. MyModel이 여전히 정의되고, 주피터 노트북 세션이 종료되지 않았기 때문에 자동 [garbage collection](https://docs.python.org/3/library/gc.html) 이 OpvModel에서 발생하지 않습니다. 그리고 OpvModel은 이전에 로드한 모델을 보유하고 있는 NCS2에 대한 참조를 보유하고 있습니다.

해당 참조를 해제하려면 다음을 수행 할 수 있습니다.:
1. mymodel.ClearMachine() 실행
2. 노트북을 중지하고 닫기
3. 사용하지 않을 경우 NCS2를 분리

안전을 위해, 과열을 방지하기 위해 사용하지 않을 때 USB 포트에서 NCS2를 제거하는 것이 좋습니다.

또한 NCS2에 대한 새로운 파이썬 API가 언제 릴리스될지를 확인합니다.

In [109]:
mymodel.ClearMachine()

## 도전과제 10: 사전 학습된 모델의 인텔 문서를 살펴보십시오. 얼마나 많은 유형의 모델을 찾을 수 있습니까?
이 모델에 대한 설명서는 %OPENVINO_INSTALL_DIR%/Intel/computer_vision_sdk/deployment_tools/
intel_models/index.html 에서 확인할 수 있습니다.

In [110]:
#Look out for the documentation!

## 유익한 시간이었나요?

여러분이 배우는 것을 잘 활용할 수 있는 방법을 생각하는 것을 잊지 마세요. 위대한 힘에는 큰 책임이 따릅니다!  여러분의 지식을 이용하여 여러분 주변의 사람들과 사회를 돕는 솔루션을 설계하고 개발하십시오!

지금까지 3개의 사전 학습된 모델만 테스트했습니다. Intel이 OpenVINO 설치와 함께 제공한 다른 많은 사전 학습 모델을 반드시 살펴보십시오!

앞으로 배우는 흥미 진진한 여행의 시작에 오신 것을 환영합니다!